In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
X = np.array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
              [1, 1, 2, 1, 3, 0, 5, 10, 1, 2],
              [500, 700, 750, 600, 1450,
               800, 1500, 2000, 450, 1000],
              [1, 1, 2, 1, 2, 
               1, 3, 3, 1, 2]], dtype = np.float64)
y = np.array([0, 0, 1, 0, 1, 0, 1, 0, 1, 1], dtype = np.float64)

In [3]:
X

array([[1.00e+00, 1.00e+00, 1.00e+00, 1.00e+00, 1.00e+00, 1.00e+00,
        1.00e+00, 1.00e+00, 1.00e+00, 1.00e+00],
       [1.00e+00, 1.00e+00, 2.00e+00, 1.00e+00, 3.00e+00, 0.00e+00,
        5.00e+00, 1.00e+01, 1.00e+00, 2.00e+00],
       [5.00e+02, 7.00e+02, 7.50e+02, 6.00e+02, 1.45e+03, 8.00e+02,
        1.50e+03, 2.00e+03, 4.50e+02, 1.00e+03],
       [1.00e+00, 1.00e+00, 2.00e+00, 1.00e+00, 2.00e+00, 1.00e+00,
        3.00e+00, 3.00e+00, 1.00e+00, 2.00e+00]])

In [4]:
y

array([0., 0., 1., 0., 1., 0., 1., 0., 1., 1.])

In [5]:
def calc_std_feat(x):
    res = (x - x.mean()) / x.std()
    return res

In [6]:
X_st = X.copy()
X_st[2, :] = calc_std_feat(X[2, :])

In [7]:
X_st

array([[ 1.        ,  1.        ,  1.        ,  1.        ,  1.        ,
         1.        ,  1.        ,  1.        ,  1.        ,  1.        ],
       [ 1.        ,  1.        ,  2.        ,  1.        ,  3.        ,
         0.        ,  5.        , 10.        ,  1.        ,  2.        ],
       [-0.97958969, -0.56713087, -0.46401617, -0.77336028,  0.97958969,
        -0.36090146,  1.08270439,  2.11385144, -1.08270439,  0.05155735],
       [ 1.        ,  1.        ,  2.        ,  1.        ,  2.        ,
         1.        ,  3.        ,  3.        ,  1.        ,  2.        ]])

## 1*. Измените  функцию calc_logloss так, чтобы нули по возможности не попадали в np.log.  

### Добавим малую величину к вектору y_pred + 1e-299

In [8]:
def calc_logloss(y, y_pred):
    y_pred =y_pred + 1e-299
    err = - np.mean(y * np.log(y_pred) + (1.0 - y) * np.log(1.0 - y_pred))
    return err

In [9]:
# Пример применения
y1 = np.array([1, 0])
y_pred1 = np.array([0.999999999999999,0])
calc_logloss(y1, y_pred1)

4.996003610813207e-16

In [10]:
def sigmoid(z):
  res = 1 / (1 + np.exp(-z))
  return res

In [11]:
def eval_model(X, y, iterations, alpha=1e-4):
    err_old = 1
    np.random.seed(42)
    W = np.random.randn(X.shape[0])
    n = X.shape[1]
    for i in range(1, iterations + 1):
        z = np.dot(W, X)
        y_pred = sigmoid(z)
        err = calc_logloss(y, y_pred)
        W -= alpha * (1/n * np.dot((y_pred - y), X.T))
        if i % (iterations / 10) == 0:
            print(i, W, err, alpha)
        if abs(err_old - err)< 1e-8:
            print('err_old - err < 1e-8 ',i, W, err, alpha)
            break
        err_old = err
    return W, err, i

In [12]:
W, err, i = eval_model(X_st, y, iterations=100000, alpha=1e-4)

10000 [ 0.25808924 -0.68189547  0.68834749  1.2411954 ] 0.5906743816929662 0.0001
20000 [ 0.14828711 -0.69312078  0.77938652  1.21304006] 0.5686161184060843 0.0001
30000 [ 0.06280025 -0.6861144   0.85151403  1.21295529] 0.5559834472247203 0.0001
40000 [-0.00866482 -0.68420916  0.90683954  1.22454952] 0.5476380224356814 0.0001
50000 [-0.06996594 -0.6866606   0.94956256  1.24413705] 0.541644374202672 0.0001
60000 [-0.12382407 -0.69204981  0.98292962  1.26931272] 0.5369564319716832 0.0001
70000 [-0.17224326 -0.69935523  1.00931153  1.29828743] 0.5330172710285223 0.0001
80000 [-0.21668343 -0.70786696  1.03042468  1.32975123] 0.5295310331283415 0.0001
90000 [-0.25820024 -0.71709221  1.04751085  1.36276018] 0.5263390430000645 0.0001
100000 [-0.29755454 -0.72668938  1.06147096  1.3966418 ] 0.5233543557317325 0.0001


## 2. Подберите аргументы функции eval_model для логистической регрессии таким образом, чтобы log loss был минимальным.

In [13]:
res=[]
e=[]
for a in range(2,8):
    alpha = 1/10**a
    print('alpha =', alpha,'#'*100 )
    W, err, i = eval_model(X_st, y, iterations=1000000, alpha=alpha)
    res.append((W,err, i, alpha))
    e.append(err)
print('__'*50)
print(f'iterations =  {res[e.index(min(e))][2]}  alpha = {res[e.index(min(e))][3]}')
Wbest = res[e.index(min(e))][0]
print('Wbest= ',Wbest)

alpha = 0.01 ####################################################################################################
100000 [-11.27192801  -1.45338517  -2.38300258   9.49385018] 0.25238285377547093 0.01
200000 [-16.2422902   -1.86937288  -3.90462901  13.50867797] 0.2087444482151381 0.01
300000 [-20.12046178  -2.20554145  -5.06338174  16.65513544] 0.18225133394128204 0.01
400000 [-23.32957684  -2.48701096  -6.00950463  19.2580126 ] 0.16415892687401012 0.01
500000 [-26.06999299  -2.72865738  -6.81029352  21.47844134] 0.1509957371068139 0.01
600000 [-28.46471856  -2.94024139  -7.50574487  23.41628013] 0.14096415738168322 0.01
700000 [-30.59526535  -3.12845003  -8.12178098  25.13788906] 0.13303809324891197 0.01
800000 [-32.51817754  -3.29802355  -8.67608723  26.68935959] 0.12659237581823718 0.01
900000 [-34.27409432  -3.4524232   -9.18120823  28.10384248] 0.12122604278612226 0.01
1000000 [-35.89309329  -3.59424249  -9.64632889  29.40588541] 0.1166708320375139 0.01
alpha = 0.001 ##############

## 3. Создайте функцию calc_pred_proba, возвращающую предсказанную вероятность класса 1 (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred_proba).

In [14]:
#Wbest = np.array([-35.89309329,  -3.59424249,  -9.64632889,  29.40588541])
def calc_pred_proba (X, W):
    z = np.dot(W, X)
    y_pred_proba = sigmoid(z)
    return y_pred_proba

In [20]:
y_pred_proba= calc_pred_proba(X_st, Wbest)
print(y_pred_proba)

[0.34705581 0.00984674 1.         0.06777614 0.93622567 0.04716302
 1.         0.01783199 0.58968717 0.99999976]


## 4. Создайте функцию calc_pred, возвращающую предсказанный класс (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred).

In [16]:
def calc_pred (X, W):
    y_pred_proba=calc_pred_proba(X,W)
    y_pred = np.round(y_pred_proba,0)
    print('y_pred =', y_pred)
    print('y      =', y)
    return y_pred

In [17]:
y_pred=calc_pred(X_st, Wbest)

y_pred = [0. 0. 1. 0. 1. 0. 1. 0. 1. 1.]
y      = [0. 0. 1. 0. 1. 0. 1. 0. 1. 1.]


## 5. Посчитайте Accuracy, матрицу ошибок, точность и полноту, а также F1 score.

In [34]:
# В качестве меры точности возьмем долю правильных ответов
_accuracy = 100.0 - np.mean(np.abs(y_pred_proba - y)*100.0)
_accuracy2 = 100.0 - np.mean(np.abs(y_pred - y)*100.0)

print(f"Итоговый вектор весов w: {Wbest}")
print(f"Точность на обучающей выборке: {_accuracy:.3f}")
print(f"Точность на обучающей выборке: {_accuracy2:.3f}")


Итоговый вектор весов w: [-35.89309329  -3.59424249  -9.64632889  29.40588541]
Точность на обучающей выборке: 90.362
Точность на обучающей выборке: 100.000


In [35]:
def EM(y, y_pred):
    em = {'TP':0,'FN':0,'FP':0,'TN':0 }
    for i in range(0, len(y)):
        if y[i] == 1 and y_pred[i]==1:
            em['TP']=  em['TP']+1
        elif y[i] == 1 and y_pred[i]==0:
            em['FN']=  em['FN']+1
        elif y[i] == 0 and y_pred[i]==1:
            em['FP']=  em['FP']+1
        elif y[i] == 0 and y_pred[i]==0:
            em['TN']=  em['TN']+1
    return em

In [36]:
em = EM(y, y_pred)
print(em)

{'TP': 5, 'FN': 0, 'FP': 0, 'TN': 5}


In [37]:
precision = em['TP']/(em['TP'] + em['FP'])
print(precision)

1.0


In [38]:
recall = em['TP']/(em['TP']+em['FN'])
print(recall)

1.0


In [39]:
F1 = 2*precision*recall/(recall+precision)
print(F1)

1.0


## 6. Могла ли модель переобучиться? Почему?

Недостаточно данных

## Дополнительно

In [40]:
from sklearn.linear_model import LogisticRegression

In [41]:
Xx=X.transpose()
Xx.shape

(10, 4)

In [42]:
y.shape

(10,)

In [43]:
clf = LogisticRegression(random_state=0).fit(Xx, y)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [44]:
clf.predict_proba(Xx)

array([[0.51295612, 0.48704388],
       [0.52135981, 0.47864019],
       [0.37036988, 0.62963012],
       [0.51715918, 0.48284082],
       [0.4473712 , 0.5526288 ],
       [0.47521519, 0.52478481],
       [0.34844531, 0.65155469],
       [0.61442591, 0.38557409],
       [0.51085387, 0.48914613],
       [0.38023253, 0.61976747]])

In [45]:
clf.coef_

array([[-2.96116774e-01, -2.01539415e-01, -1.68275878e-04,
         8.26074825e-01]])

In [46]:
clf.intercept_

array([-0.29611677])

In [47]:
clf.score(Xx,y)

0.8